# マチンの公式

以下の円周率公式はマチンの公式と呼ばれています。

$$\frac{\pi}{4} = 4 \arctan \frac{1}{5} - \arctan \frac{1}{239} = \sum_{n=0}^{\infty} \frac{(-1)^n}{2n+1} \left(\frac{1}{5}\right)^{2n+1} -\sum_{n=0}^{\infty} \frac{(-1)^n}{2n+1} \left(\frac{1}{239}\right)^{2n+1}$$

この公式が正しいことは、$\arctan$ の加法定理

$$\arctan a \pm \arctan b = \arctan \frac{a \pm b}{1 \mp ab} \quad (ab < 1)$$

を用いれば証明できます。この公式を用いて円周率を計算してみましょう。まずは単純に、それぞれの級数を同じ項まで計算してみましょう。しかし符号の問題で、

$$\begin{align}
&4 \cdot \frac{1}{5} -\left(\frac{4}{3}\frac{1}{5^3} +\frac{1}{239}\right) + \left(\frac{4}{5}\frac{1}{5^5} +\frac{1}{3}\frac{1}{239^3}\right) -\cdots - \left(\frac{4}{4k-1} \frac{1}{5^{4k-1}} + \frac{1}{4k-3}\frac{1}{239^{4k-3}}\right)\\
&\leq  \frac{\pi}{4} \leq \\
& 4 \cdot \frac{1}{5} -\left(\frac{4}{3}\frac{1}{5^3} +\frac{1}{239}\right) +\left(\frac{4}{5}\frac{1}{5^5} +\frac{1}{3}\frac{1}{239^3}\right) -\cdots + \left(\frac{4}{4k+1} \frac{1}{5^{4k+1}} + \frac{1}{4k-1}\frac{1}{239^{4k-1}}\right)
\end{align}$$

が成り立つので、$\frac{1}{239}$ の級数の項は 1 少なくします。

In [2]:
from typing import Generator, Tuple
from dataclasses import dataclass
import mpmath

@dataclass
class ArctanParam:
    coeff: int
    numer: int  # 分子
    denom: int  # 分母
        
    def __post_init__(self):
        if self.coeff <= 0:
            raise ValueError(f"coeff must be greater than 0. given {self.coeff}")
        if self.numer <= 0:
            raise ValueError(f"numer must be greater than 0. given {self.numer}")
        if self.denom <= 0:
            raise ValueError(f"denom must be greater than 0. given {self.denom}")

def arctan_series(p: ArctanParam) -> Generator[Tuple[float, int], None, None]:
    def next_term(cur_term: float, x: float, n: int) -> float:
        # a_{n+1} を返す
        return (-1) * (mpmath.mpf(2 * n + 1) / (2 * n + 3)) * cur_term * (x ** 2)
    
    n = 0
    
    x = mpmath.mpf(p.numer) / p.denom
    cur_term = p.coeff * x
    _sum = cur_term
    
    while True:
        yield (_sum, n)
        cur_term = next_term(cur_term, x, n)
        _sum = _sum + cur_term
        n = n + 1

In [3]:
import sys
sys.path.append('/home/jovyan/work/')
from lib.utils import print_pi, PI_50

DPS = 20
mpmath.mp.dps = DPS

param_5 = ArctanParam(16, 1, 5)
param_239 = ArctanParam(4, 1, 239)

at_5 = arctan_series(param_5)
at_239 = arctan_series(param_239)

prev_pi = 0
(pi, _) = next(at_5)

print(f"π = {PI_50}...\n")

for i in range(1, 11):
    
    (a5, _) = next(at_5)
    (a239, _) = next(at_239)

    prev_pi = pi
    pi = a5 - a239
    #print(f"16 arctan 1/5 \t= {a5}")
    #print(f"4 arctan 1/239 \t= {a239}")
    print(f"n = {i}:", end="\t")
    print_pi(pi, PI_50, DPS, _format="{pi} ({mdigit} 桁まで一致)")
    """
    if i % 2 == 1:
        print(f"{pi} ≦ π ≦ {prev_pi}")
    else:
        print(f"{prev_pi} ≦ π ≦ {pi}")
    """

π = 3.14159265358979323846264338327950288419716939937510...

n = 1:	3.1405969316596931660 (2 桁まで一致)
n = 2:	3.1416210293260603143 (3 桁まで一致)
n = 3:	3.1415917721821772822 (5 桁まで一致)
n = 4:	3.1415926824043995172 (7 桁まで一致)
n = 5:	3.1415926526153086081 (8 桁まで一致)
n = 6:	3.1415926536235547620 (9 桁まで一致)
n = 7:	3.1415926535886022287 (11 桁まで一致)
n = 8:	3.1415926535898358475 (12 桁まで一致)
n = 9:	3.1415926535897916969 (14 桁まで一致)
n = 10:	3.1415926535897932947 (16 桁まで一致)


## $\arctan$ の誤差

実は $\arctan \frac{1}{239}$ の方が速く収束するので、同じ項数計算するのは効率が悪いです。どのように計算するのが効率が良いか考えてみましょう。$\arctan$ のテイラー展開

$$\arctan x = x -\frac{1}{3}x^3 + \frac{1}{5} x^5 - \cdots = \sum_{n = 0}^{\infty} \frac{(-1)^n}{2n+1} x^{2n+1} \quad (|x| \leq 1)$$

を $k -1$ 項まででとめて、

$$\begin{align}
\left|\arctan x -\sum_{n=0}^{m} \frac{(-1)^n}{2n+1}x^{2n+1}\right| = E_m(x) \\
\end{align}$$

とおきます。$x < 1$ のときに

$$\frac{1}{1 + x^2} = 1 -x^2 + x^4 - \cdots + (-1)^{m}x^{2m} -\frac{(-1)^{m}x^{2m+2}}{1 +x^2}$$

から、

$$\arctan x = \int_0^x \frac{1}{1 + t^2} dt = \sum_{n=0}^{m} \frac{(-1)^n}{2n+1}x^{2n+1} +\int_0^x \frac{(-1)^{m+1}t^{2m+2}}{1 +t^2} dt$$

なので、

$$E_m(x) = \left|\int_0^x \frac{t^{2m+2}}{1 +t^2} dt \right|$$

が成り立ちます。ここで、$$1 \leq 1 + t^2 \leq 1 +x^2$$ から

$$\frac{1}{1 +x^2} \frac{|x^{2m+3}|}{2m+3} \leq \frac{1}{1 +x^2} \left| \int_0^x t^{2m+2} dt \right| \leq E_m(x) \leq \left| \int_0^x t^{2m+2} dt \right| = \frac{|x^{2m+3}|}{2m+3}$$

が成り立ちます。$x$ が小さければ $\frac{1}{1 +x^2}$ は $1$ に近いので、$E_m(x)$ はおおよそ $\frac{|x^{2m+3}|}{2m+3}$ であると考えられます。例えば $x = \frac{1}{5}$ のときは $\frac{1}{1 +x^2} = \frac{100}{104} = 0.96153...$ です。$x$ が小さければ小さいほど誤差は小さくなります。

## マチンの公式の誤差

マチンの公式には $\arctan$ の項が 2 つあるので、$x = \frac{1}{5}$ の項を $m_1$ まで、$x = \frac{1}{239}$ の項を $m_2$ まで計算したときの誤差を計算しましょう。少し粗いですが、

$$\begin{align}
& \left| \pi - 4\left(4\sum_{n=0}^{m_1} \frac{(-1)^n}{2n+1} \frac{1}{5^{2n+1}} -\sum_{n=0}^{m_2} \frac{(-1)^n}{2n+1} \frac{1}{239^{2n+1}}\right) \right| \\
= \ &  \left|16E_{m_1}\left(\frac{1}{5}\right) + 4E_{m_2}\left(\frac{1}{239}\right)\right| \\
\leq  \ & 16\left|E_{m_1}\left(\frac{1}{5}\right)\right| + 4\left|E_{m_2}\left(\frac{1}{239}\right)\right|  \\
\leq \ & 16\frac{1}{(2m_1+3)5^{2m_1+3}} + 4\frac{1}{(2m_2+3)239^{2m_2+3}}
\end{align}$$

と評価します。$\pi$ を100桁まで求めるには、誤差をおおよそ $10^{-100}$ にすれば良いです (十分ではありません)。 そのために $m_1$, $m_2$ をどのように選べば良いか概算します。$m_1$ と $m_2$ は、片方だけ大きくしていっても、もう片方の誤差が減らないので、

$$16\left|E_{m_1}\left(\frac{1}{5}\right)\right| \fallingdotseq 4\left|E_{m_2}\left(\frac{1}{239}\right)\right|$$

を満たすように選ぶと良さそうです。誤差をおおよそ $10^{-100}$  にするので、

$$16\left|E_{m_1}\left(\frac{1}{5}\right)\right| + 4\left|E_{m_2}\left(\frac{1}{239}\right)\right| \fallingdotseq 10^{-100}$$

になるようにすると、$\left|E_{m_1}\left(\frac{1}{5}\right)\right| \fallingdotseq \frac{1}{(2m_1+3)5^{2m_1+3}}$ とみなして

$$\begin{align}
& 32 \left|E_{m_1}\left(\frac{1}{5}\right)\right| \leq 10^{-100} \\
\Leftrightarrow \ & \frac{1}{(2m_1+3)5^{2m_1+3}} \leq \frac{1}{32} 10^{-100} \\
\Leftrightarrow \ & (2m_1+3)5^{2m_1+3} \geq 32 \cdot 10^{100} \\
\Leftrightarrow \ & \log_{10}(2m_1 + 3) + (2m_1 + 3) \log_{10}5 \geq 100 + \log_{10} 32
\end{align}$$

を満たす最小の $m_1$ を取ります。同様に、

$$\begin{align}
& 8 \left|E_{m_1}\left(\frac{1}{239}\right)\right| \leq 10^{-100} \\
\Leftrightarrow \ & \log_{10}(2m_2 + 3) + (2m_2 + 3) \log_{10}239 \geq 100 + \log_{10} 8
\end{align}$$

を満たす最小の $m_2$ を取ります。$m_1$, $m_2$ を計算してみましょう。

In [9]:
from math import log10
LOOP_LIMIT = 1000

param_5 = ArctanParam(16, 1, 5)
param_239 = ArctanParam(4, 1, 239)

def arctan_log_err(p: ArctanParam, n: int) -> float:
    return log10(p.coeff) - log10(2 * n + 3) + (2 * n + 3) * log10(p.numer / p.denom)

def calc_num_of_term(p: ArctanParam, log_err: float) -> int:
    n = 1
    while arctan_log_err(p, n) > log_err and n < LOOP_LIMIT:
        n = n + 1
    if n == LOOP_LIMIT:
        raise Error("ループ回数が多い")
    return n

m1 = calc_num_of_term(param_5, -100 - log10(2))
m2 = calc_num_of_term(param_239, -100 - log10(2))

print(f"m1 = {m1}, m2 = {m2}")
print(f"Em1 = {arctan_log_err(param_5, m1)}, Em2 = {arctan_log_err(param_239, m2)}")

param_7 = ArctanParam(20, 1, 7)
param_79 = ArctanParam(8, 3, 79)

m1 = calc_num_of_term(param_7, -20 - log10(2))
m2 = calc_num_of_term(param_79, -20 - log10(2))

print(f"m1 = {m1}, m2 = {m2}")
print(f"Em1 = {arctan_log_err(param_7, m1)}, Em2 = {arctan_log_err(param_79, m2)}")

m1 = 70, m2 = 20
Em1 = -100.90392667485982, Em2 = -103.30251820502156
m1 = 11, m2 = 6
Em1 = -21.22436101336448, Em2 = -21.580588820625422


## 100桁までの計算

$m_1 = 70$, $m_2 = 20$ と求まったので、その付近まで計算してみましょう。

In [46]:
import sys
sys.path.append('/home/jovyan/work/')
from lib.utils import print_pi

DPS = 150
mpmath.mp.dps = DPS

PI_150 = "3.14159265358979323846264338327950288419716939937510" \
                    + "58209749445923078164062862089986280348253421170679" \
                    + "82148086513282306647093844609550582231725359408128"

def error(m: int, p: ArctanParam):
    x = mpmath.mpf(p.numer)  / p.denom
    return p.coeff * (mpmath.mpf(1) / (2 * m + 3)) * x ** (2 * m + 3)
    
# 準備
param_5 = ArctanParam(16, 1, 5)
param_239 = ArctanParam(4, 1, 239)

at_5 = arctan_series(param_5)
at_239 = arctan_series(param_239)

for i in range(0, 70):
    next(at_5)
for i in range(0, 19):
    next(at_239)
    
(a5, m1) = next(at_5)
(a239, m2) = next(at_239)
# 準備終わり

def print_result(a5, a239, m1, m2):
    pi = a5 - a239

    e_1 = error(m1, param_5)
    e_2 = error(m2, param_239)

    E = e_1 + e_2

    print(f"m1 = {m1}, m2 = {m2}")

    print("・誤差無視")
    print_pi(pi, PI_150, DPS, _format="{pi} ({mdigit} 桁まで一致)")
    print("")

    print("・誤差考慮")
    print_pi(pi - E, PI_150, DPS, _format="{pi} ({mdigit} まで一致)")
    print(" ≦ π ≦ ")
    print_pi(pi + E, PI_150, DPS, _format="{pi} ({mdigit} まで一致)")
    print("")
    
    print("・誤差")
    print(E)
    
print_result(a5, a239, m1, m2)

(a239_next, m2_next) = next(at_239)
(a5_next, m1_next) = next(at_5)

print("\n========\n")
print_result(a5, a239_next, m1, m2_next)

print("\n========\n")
print_result(a5_next, a239, m1_next, m2)

print("\n========\n")
print_result(a5_next, a239_next, m1_next, m2_next)

m1 = 70, m2 = 19
・誤差無視
3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211707097922348653033942265534569142226838792174580775170 (97 桁まで一致)

・誤差考慮
3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706798162478513934506898440362801792706010179987216807 (101 まで一致)
 ≦ π ≦ 
3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211707397682218792133377632628775482660971574169174333533 (97 まで一致)

・誤差
2.99759870139099435367094206340434132781994593558363246127089703246251336712369789921706362296060219749821664262343719932840839257016545920534024086986e-99


m1 = 70, m2 = 20
・誤差無視
3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706799410072650915241060204598789073733227297772727296 (100 桁まで一致)

・誤差考慮
3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170679815749561825524

これで $m_1 = 71$, $m_2 = 20$ まで計算すれば小数点以下100桁まで求まることがわかりました。$m_1, m_2$ は $0$ から始まるので、$72 + 21 = 93$ 項計算すれば100桁まで求まります。

最後にアルキメデスの方法で何角形まで求めなければならないか計算してみましょう。

In [8]:
import mpmath

def geom_mean(a: float, b: float) -> float:
    return mpmath.sqrt(a * b)

def harmonic_mean(a: float, b: float) -> float:
    return 2*a*b / (a + b)

def _next(a: float, b: float) -> Tuple[float]:
    b_ = harmonic_mean(a, b)
    return (
        geom_mean(a, b_),
        b_
    )

A0, B0 = (2 * mpmath.sqrt(2), mpmath.mpf(4))

In [13]:
A = A0
B = B0

i = 0

while B - A > 10 ** -100 or i > 10000:
    A, B = _next(A, B)
    i = i + 1
    
print(i)
print_pi(A, PI_150, DPS)
print_pi(A, PI_150, DPS)

167
π = 3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706797291897627989888769952974788109009512665656597476 (100 桁まで一致)
π = 3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706797291897627989888769952974788109009512665656597476 (100 桁まで一致)


よって $6 \cdot 2^{167} = 1122433257470133441180429951526105359095756193005568$ 角形まで計算する必要があります。